# Langchain Get Started tutorial

LangChain is a framework for developing applications powered by language models

- Lang: Stands for language, which is the primary focus of LangChain
- Chain:  the connotation of connecting things 

## 1. Installation

This will install the bare minimum requirements of LangChain. if you are gonna request to opeanAI models, it must be to install openai library

In [ ]:
#!pip install langchain

## 2. Environment setup

Depend on your LLM provider, it's necessary to install their Python package, E.g: we're gonna use openAI models:

In [ ]:
#!pip install openai

let's create a .env file with credentials

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

or set within notebook.

In [ ]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "<>"
os.environ["OPENAI_API_BASE"] = "..."
os.environ["OPENAI_API_KEY"] = "..."

Note: In this example we're are using azureOpenAI services

## 3. Some concepts

The most common and most important chain that LangChain helps create contains three things:

- **LLM:** The language model is the core reasoning engine here 

- **Prompt Templates:** This provides instructions to the language model. This controls what the language model outputs

- **Output Parsers:** These translate the raw response from the LLM to a more workable format, making it easy to use the output downstream.

Most LangChain applications allow you to configure the LLM and/or the prompt used

## A) LLMs

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*TZbfdCyPRLi9ZAORn6WnGw.png" width="500" height="200" />


There are two types of language models, which in LangChain are called:

- **LLMs:** this is a language model which takes a string as input and returns a string
    - input: String
    - output: String
- **ChatModels:** this is a language model which takes a list of messages as input and returns a message
    - input: List[ChatMessages]
    - output: single ChatMessage


**ChatMessage**: has two required components:

- **role:** This is the role of the entity from which the ChatMessage is coming from.

    - **HumanMessage:** A ChatMessage coming from a human/user.
    - **AIMessage:** A ChatMessage coming from an AI/assistant.
    - **SystemMessage:** A ChatMessage coming from the system.
    - **FunctionMessage:** A ChatMessage coming from a function call.
  


- **content:** This is the content of the message

The standard interface that LangChain provides has two methods:

- predict: Takes in a string, returns a string
- predict_messages: Takes in a list of messages, returns a message.

In [ ]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

In [ ]:
llm = AzureOpenAI(temperature=0.0, deployment_name= "chatGPTInstruct", model_name="gpt-35-turbo-instruct", verbose=False)
chat_model = AzureChatOpenAI(deployment_name=  "GPT35-16k", model_name="gpt-35-turbo-16k")

In [ ]:
text = "What would be a good company name for a company that makes colorful socks?"

In [ ]:
llm.predict(text) # Try to predict next words

![llm completions](../../resources/llm_completion.png)

In [ ]:
chat_model.predict(text) # Try to answer the question

Finally, let's use the predict_messages method

In [ ]:
from langchain.schema import HumanMessage

In [ ]:
text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]
messages

In [ ]:
llm.predict_messages(messages)

In [ ]:
chat_model.predict_messages(messages)

## B) Prompt templates

The LLM applications do not pass user input directly into an LLM.

**Prompt template:** Provides additional context on the specific task at hand.

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")

In [ ]:
prompt.format(product="colorful socks")

In [ ]:
prompt.format(product="food")

PromptTemplates can also be used to produce a list of messages

The prompt not only contains information about the content, but also each message (its role, its position in the list, etc.). 

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

In [ ]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([ #List of messages with structure role:..., content
    ("system", template), # system role
    ("human", human_template), # human, the user text
])

In [ ]:
chat_prompt

In [ ]:
messages = chat_prompt.format_messages(input_language="English",
                            output_language="Spanish",
                           text= "Langchain is awesome")

In [ ]:
messages

In [ ]:
chat_model.predict_messages(messages)

In [ ]:
llm.predict_messages(messages)

## C) Output parsers

OutputParsers convert the raw output of an LLM into a format that can be used downstream, There are few main types of OutputParsers, including:

- Convert text from LLM into structured information (e.g. JSON)
- Convert a ChatMessage into just a string
- Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.


Let's write our own output parser - one that converts a comma separated list into a list.

In [ ]:
from langchain.schema import BaseOutputParser

In [ ]:
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

In [ ]:
CommaSeparatedListOutputParser().parse("papa, yuca, platano, maracuya")

## Example using: PromptTemplate + LLM + OutputParser [NOT CHAIN, manually]

### 1. Create ChatPromptTemplate 

In [ ]:
template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a country, and you should generate 5 food of that country in a comma separated list.
ONLY return a comma separated list, and nothing more."""

human_template = "{country}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template), 
    ("human", human_template), 
])

### 2. Format ChatPromptTemplate 

In [ ]:
messages = chat_prompt.format_messages(country="colombia")

messages

### 3. Predict messages using ChatPromptTemplate 

In [ ]:
food = chat_model.predict_messages(messages).content

food

### 4. Output Parser

In [ ]:
CommaSeparatedListOutputParser().parse(food)

## Example using: PromptTemplate + LLM + OutputParser [CHAIN]

just two code lines to do the task

In [ ]:
chain = chat_prompt | chat_model | CommaSeparatedListOutputParser()

In [ ]:
chain.invoke({"country":"Colombia"})

In [ ]:
chain.batch([{"input":"Colombia"},{"input":"Argentina"}])